In [ ]:
'''
Created on Nov 4, 2010
Chapter 5 source file for Machine Learing in Action
@author: Peter
'''
from numpy import *
from time import sleep

In [ ]:
def loadDataSet(fileName):
    # 初始化数据和标签列表
    dataMat = []; labelMat = []
    
    # 打开指定文件，返回文件对象
    fr = open(fileName)
    
    # 对文件中的每一行进行迭代
    for line in fr.readlines():
        # 剔除字符串首尾的空格并按照 '\t' 分割字符串，存入列表
        lineArr = line.strip().split('\t')
        
        # 将前两个元素转换为浮点数并添加到数据列表中
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        
        # 将第三个元素转换为浮点数并添加到标签列表中
        labelMat.append(float(lineArr[2]))
        
    # 返回数据和标签列表
    return dataMat, labelMat


In [ ]:
def selectJrand(i, m):
    """
    随机选择一个不等于i的整数j，其中0<=j<=m-1
    
    Args:
    i: int, 当前的i值
    m: int, 选择j时的上界
    
    Returns:
    j: int, 随机选择的不等于i的整数
    """
    j = i  # 我们想要选择一个不等于i的任何j
    
    # 当j等于i时，随机选择一个0到m-1之间的整数j
    while (j == i):
        j = int(random.uniform(0, m))
    
    # 返回随机选择的不等于i的整数j
    return j


In [ ]:
def clipAlpha(aj, H, L):
    """
    将alpha[j]调整到L和H之间的范围内
    
    Args:
    aj: float, 要调整的alpha[j]值
    H: float, alpha[j]的上界
    L: float, alpha[j]的下界
    
    Returns:
    aj: float, 调整后的alpha[j]值
    """
    # 如果alpha[j]大于上界H，则将其设置为上界H
    if aj > H:
        aj = H
    
    # 如果alpha[j]小于下界L，则将其设置为下界L
    if L > aj:
        aj = L
    
    # 返回调整后的alpha[j]值
    return aj


In [ ]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn)  # 将特征数据转换为矩阵
    labelMat = mat(classLabels).transpose()  # 将标签数据转换为矩阵并进行转置
    b = 0  # 初始化偏置值为0
    m,n = shape(dataMatrix)  # 获取特征矩阵的大小，m：样本数，n：属性个数
    alphas = mat(zeros((m,1)))  # 初始化拉格朗日乘子向量alpha为全0的列向量
    iter = 0  # 初始化迭代次数为0
    while (iter < maxIter):  # 迭代开始
        alphaPairsChanged = 0  # 每一轮迭代前重置alpha对的改变次数为0
        for i in range(m):  # 遍历每一个样本
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b  # 计算当前样本的预测值f(x_i)
            Ei = fXi - float(labelMat[i])  # 计算误差Ei
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):  # 检查是否违反KKT条件
                j = selectJrand(i,m)  # 随机选择另一个样本j
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b  # 计算样本j的预测值f(x_j)
                Ej = fXj - float(labelMat[j])  # 计算误差Ej
                alphaIold = alphas[i].copy()  # 备份alpha_i旧值
                alphaJold = alphas[j].copy()  # 备份alpha_j旧值
                if (labelMat[i] != labelMat[j]):  # 判断两个样本标签是否相同
                    L = max(0, alphas[j] - alphas[i])  # 计算L和H的值
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print("L==H"); continue  # 如果L和H相等，则退出当前循环，继续下一次循环
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T  # 计算eta的值
                if eta >= 0: print("eta>=0"); continue  # 如果eta大于等于0，则退出当前循环，继续下一次循环
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta  # 更新alpha_j的值
                alphas[j] = clipAlpha(alphas[j],H,L)  # 调整alpha_j的值在[L, H]区间范围内
                if (abs(alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); continue  # 如果alpha_j的变化量太小，则退出当前循环，继续下一次循环
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])  # 更新alpha_i的值
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T  # 计算b1和b2的值
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1  # 更新偏置值b
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1  # alpha对的改变次数加1
                print("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))  # 打印当前迭代次数、样本编号和alpha对的改变次数
        if (alphaPairsChanged == 0): iter += 1  # 如果alpha对的改变次数为0，则迭代次数加1
        else: iter = 0
        print("iteration number: %d" % iter)  # 打印当前迭代次数
    return b,alphas

In [ ]:
def kernelTrans(X, A, kTup):
    """
    计算核函数或将数据转换到高维空间
    
    Args:
    X: matrix, 输入的特征矩阵
    A: matrix, 用于计算核函数的向量
    kTup: tuple, 核函数的类型及相关参数
    
    Returns:
    K: matrix, 计算得到的核矩阵
    """
    m, n = shape(X)  # 获取特征矩阵的维度，m为样本数，n为属性个数
    K = mat(zeros((m, 1)))  # 初始化核矩阵为0
    
    if kTup[0] == 'lin':
        K = X * A.T  # 线性核函数
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K / (-1 * kTup[1] ** 2))  # 在NumPy中除法是逐元素进行的，而不是矩阵运算（类似于Matlab）
    else:
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')  # 未识别的核函数类型
    
    return K


In [ ]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        """
        初始化优化结构体
        
        Args:
        dataMatIn: matrix, 输入的特征矩阵
        classLabels: matrix, 输入的类别标签
        C: float, 松弛变量
        toler: float, 容错率
        kTup: tuple, 核函数的类型及相关参数
        """
        self.X = dataMatIn  # 输入的特征矩阵
        self.labelMat = classLabels  # 输入的类别标签
        self.C = C  # 松弛变量
        self.tol = toler  # 容错率
        self.m = shape(dataMatIn)[0]  # 样本数
        self.alphas = mat(zeros((self.m, 1)))  # 拉格朗日乘子向量
        self.b = 0  # 截距
        self.eCache = mat(zeros((self.m, 2)))  # 误差缓存，第一列为有效标志位
        self.K = mat(zeros((self.m, self.m)))  # 核矩阵
        
        # 计算核矩阵
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)


In [ ]:
def calcEk(oS, k):
    """
    计算样本k的预测误差
    
    Args:
    oS: optStruct, 优化结构体
    k: int, 样本的索引
    
    Returns:
    Ek: float, 样本k的预测误差
    """
    fXk = float(multiply(oS.alphas, oS.labelMat).T * oS.K[:, k] + oS.b)  # 样本k的预测结果
    Ek = fXk - float(oS.labelMat[k])  # 样本k的预测误差
    return Ek


In [ ]:
def selectJ(i, oS, Ei):
    """
    选择第二个变量alpha_j并计算其预测误差Ej
    
    Args:
    i: int, 第一个变量alpha_i的索引
    oS: optStruct, 优化结构体
    Ei: float, 第一个变量alpha_i的预测误差
    
    Returns:
    j: int, 第二个变量alpha_j的索引
    Ej: float, 第二个变量alpha_j的预测误差
    """
    maxK = -1  # 保存选择的变量alpha_j的索引
    maxDeltaE = 0  # 保存选择的变量alpha_j的预测误差与alpha_i预测误差之差的绝对值
    Ej = 0  # 保存选择的变量alpha_j的预测误差
    oS.eCache[i] = [1, Ei]  # 设置alpha_i的误差缓存为有效
    validEcacheList = nonzero(oS.eCache[:, 0].A)[0]  # 获取所有有效的误差缓存的索引列表
    if len(validEcacheList) > 1:  # 如果有多个有效的误差缓存值
        for k in validEcacheList:  # 遍历有效的误差缓存值，找到使得delta E最大的alpha_j
            if k == i:
                continue  # 不计算alpha_i对应的误差，浪费时间
            Ek = calcEk(oS, k)  # 计算alpha_k的预测误差
            deltaE = abs(Ei - Ek)  # 计算预测误差之差的绝对值
            if deltaE > maxDeltaE:  # 找到使得delta E最大的alpha_j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:  # 如果是第一次循环，没有有效的误差缓存值
        j = selectJrand(i, oS.m)  # 随机选择一个不等于i的索引值作为alpha_j的索引
        Ej = calcEk(oS, j)  # 计算alpha_j的预测误差
    return j, Ej


In [ ]:
def updateEk(oS, k):
    """
    更新误差缓存中样本k的预测误差值
    
    Args:
    oS: optStruct, 优化结构体
    k: int, 样本的索引
    """
    Ek = calcEk(oS, k)  # 计算样本k的预测误差
    oS.eCache[k] = [1, Ek]  # 更新误差缓存中样本k的预测误差值


In [ ]:
def innerL(i, oS):
    """
    内循环中的优化过程，选择并更新两个变量alpha_i和alpha_j
    
    Args:
    i: int, 第一个变量alpha_i的索引
    oS: optStruct, 优化结构体
    
    Returns:
    1: 表示alpha_i和alpha_j已更新
    0: 表示alpha_i和alpha_j未更新
    """
    Ei = calcEk(oS, i)  # 计算第一个变量alpha_i的预测误差
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        # 检查是否违反KKT条件，需要进行优化
        j, Ej = selectJ(i, oS, Ei)  # 选择第二个变量alpha_j
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L==H")
            return 0
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]  # 计算eta
        if eta >= 0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta  # 更新alpha_j
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)  # 调整alpha_j的取值范围
        updateEk(oS, j)  # 更新alpha_j的误差缓存
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])  # 更新alpha_i
        updateEk(oS, i)  # 更新alpha_i的误差缓存
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] - oS.labelMat[j] * (
                    oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j] - oS.labelMat[j] * (
                    oS.alphas[j] - alphaJold) *oS.K[j, j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
            return 1 # 表示alpha_i和alpha_j已更新
    else: 
        return 0 # 表示alpha_i和alpha_j未更新


In [ ]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    """
    Platt SMO算法的外循环函数
    
    Args:
    dataMatIn: array-like, 输入数据矩阵
    classLabels: array-like, 类别标签
    C: float, 软间隔参数
    toler: float, 容错率
    maxIter: int, 最大迭代次数
    kTup: tuple, 核函数的类型和参数
    
    Returns:
    b: float, 模型的偏置项
    alphas: array-like, 模型的拉格朗日乘子
    """
    oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler, kTup)  # 初始化优化结构体
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:  # 遍历所有样本
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)  # 进行优化
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        else:  # 遍历非边界样本
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)  # 进行优化
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False  # 切换到非边界样本循环
        elif (alphaPairsChanged == 0):
            entireSet = True  # 切换到全样本循环
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas


In [ ]:
def calcWs(alphas, dataArr, classLabels):
    """
    计算模型的回归系数w
    
    Args:
    alphas: array-like, 拉格朗日乘子
    dataArr: array-like, 输入数据矩阵
    classLabels: array-like, 类别标签
    
    Returns:
    w: array-like, 回归系数
    """
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m, n = shape(X)
    w = zeros((n, 1))
    for i in range(m):
        w += multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w


In [ ]:
def testRbf(k1=1.3):
    """
    测试使用RBF核函数的支持向量机模型
    
    Args:
    k1: float, RBF核函数的参数，默认为1.3
    
    Returns:
    None
    """
    # 加载数据集
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    # 调用SMO算法训练模型
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    # 获取支持向量的索引
    svInd = nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]  # 仅包含支持向量的数据矩阵
    labelSV = labelMat[svInd]  # 支持向量的类别标签
    print("There are %d Support Vectors" % shape(sVs)[0])
    
    m, n = shape(datMat)
    errorCount = 0
    # 在训练集上进行预测并计算训练误差
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))  # 计算RBF核函数值
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b  # 根据支持向量、乘子和偏置进行预测
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("The training error rate is: %f" % (float(errorCount) / m))
    
    # 加载测试数据集
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m, n = shape(datMat)
    # 在测试集上进行预测并计算测试误差
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))  # 计算RBF核函数值
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b  # 根据支持向量、乘子和偏置进行预测
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("The test error rate is: %f" % (float(errorCount) / m))


In [ ]:
def img2vector(filename):
    """
    将图像转换为向量表示
    
    Args:
    filename: str, 图像文件名
    
    Returns:
    returnVect: array, 转换后的图像向量，形状为(1, 1024)
    """
    returnVect = zeros((1, 1024))  # 创建一个全零向量，用于存储图像像素值
    fr = open(filename)  # 打开图像文件
    for i in range(32):  # 读取图像的每一行
        lineStr = fr.readline()  # 读取一行图像数据
        for j in range(32):  # 遍历每个像素点
            returnVect[0, 32 * i + j] = int(lineStr[j])  # 将像素值转换为整数并存储到向量中
    return returnVect  # 返回转换后的图像向量


In [ ]:
def loadImages(dirName):
    """
    加载图像数据集
    
    Args:
    dirName: str, 数据集所在的目录名
    
    Returns:
    trainingMat: array, 训练样本矩阵，每一行表示一个图像的向量表示
    hwLabels: list, 图像标签列表，每个标签表示对应图像的类别（1或-1）
    """
    from os import listdir
    
    hwLabels = []  # 存储图像标签的列表
    trainingFileList = listdir(dirName)  # 加载训练数据集目录下的文件列表
    m = len(trainingFileList)  # 训练样本的数量
    trainingMat = zeros((m, 1024))  # 创建一个全零矩阵，用于存储训练样本的向量表示
    
    for i in range(m):
        fileNameStr = trainingFileList[i]  # 获取文件名
        fileStr = fileNameStr.split('.')[0]  # 去掉文件名的扩展名
        classNumStr = int(fileStr.split('_')[0])  # 提取图像的类别标签
        if classNumStr == 9:
            hwLabels.append(-1)  # 如果图像类别为9，则标签为-1
        else:
            hwLabels.append(1)  # 否则标签为1
        trainingMat[i, :] = img2vector('%s/%s' % (dirName, fileNameStr))  # 将图像转换为向量表示，并存储到训练样本矩阵中
        
    return trainingMat, hwLabels  # 返回训练样本矩阵和图像标签列表


In [ ]:
def testDigits(kTup=('rbf', 10)):
    """
    测试手写数字识别系统
    
    Args:
    kTup: tuple, 核函数的类型和参数，默认为('rbf', 10)
    
    Returns:
    None
    """
    dataArr, labelArr = loadImages('trainingDigits')  # 加载训练数据集
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)  # 使用SMO算法训练SVM模型
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A > 0)[0]  # 获取支持向量的索引
    sVs = datMat[svInd]  # 获取支持向量对应的训练样本
    labelSV = labelMat[svInd]  # 获取支持向量对应的标签
    print("There are %d Support Vectors" % shape(sVs)[0])
    m, n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)  # 计算训练样本与支持向量之间的核函数值
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b  # 使用核函数进行预测
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("The training error rate is: %f" % (float(errorCount) / m))
    
    dataArr, labelArr = loadImages('testDigits')  # 加载测试数据集
    errorCount = 0
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m, n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)  # 计算测试样本与支持向量之间的核函数值
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b  # 使用核函数进行预测
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("The test error rate is: %f" % (float(errorCount) / m))


In [ ]:
class optStructK:
    def __init__(self, dataMatIn, classLabels, C, toler):
        """
        初始化非核版本的优化结构体

        Args:
        dataMatIn: array-like, 特征数据集
        classLabels: array-like, 类别标签
        C: float, 软间隔参数
        toler: float, 容错率

        Returns:
        None
        """
        self.X = dataMatIn  # 特征数据集
        self.labelMat = classLabels  # 类别标签
        self.C = C  # 软间隔参数
        self.tol = toler  # 容错率
        self.m = shape(dataMatIn)[0]  # 数据集的样本数
        self.alphas = mat(zeros((self.m, 1)))  # 存储拉格朗日乘子
        self.b = 0  # 分类器的偏置项
        self.eCache = mat(zeros((self.m, 2)))  # 误差缓存，第一列为有效标志位



In [ ]:
def calcEkK(oS, k):
    """
    计算第k个样本的预测误差

    Args:
    oS: optStructK对象，优化结构体
    k: int，样本索引

    Returns:
    Ek: float，预测误差
    """
    fXk = float(multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b  # 计算f(x_k)
    Ek = fXk - float(oS.labelMat[k])  # 计算预测误差Ek
    return Ek


In [ ]:
def selectJK(i, oS, Ei):
    """
    选择第二个变量和计算对应的预测误差Ej

    Args:
    i: int，第一个变量的索引
    oS: optStruct对象，优化结构体
    Ei: float，第一个变量的预测误差

    Returns:
    maxK: int，选择的第二个变量的索引
    Ej: float，第二个变量的预测误差
    """
    maxK = -1  # 保存选择的第二个变量的索引
    maxDeltaE = 0  # 保存最大的预测误差变化值
    Ej = 0  # 保存第二个变量的预测误差
    oS.eCache[i] = [1, Ei]  # 设置第一个变量的缓存为有效值

    # 获取缓存中有效的E值的索引列表
    validEcacheList = nonzero(oS.eCache[:, 0].A)[0]

    if len(validEcacheList) > 1:
        # 遍历有效的E值列表，选择使得预测误差变化值最大的第二个变量
        for k in validEcacheList:
            if k == i:
                continue  # 不计算第一个变量的预测误差
            Ek = calcEk(oS, k)  # 计算第二个变量的预测误差
            deltaE = abs(Ei - Ek)  # 计算预测误差的变化值
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        # 如果缓存中没有有效的E值（第一次迭代），则随机选择第二个变量
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej


In [ ]:
def updateEkK(oS, k):
    """
    更新缓存中第k个样本的预测误差Ek

    Args:
    oS: optStruct对象，优化结构体
    k: int，样本索引

    Returns:
    None
    """
    Ek = calcEk(oS, k)  # 计算第k个样本的预测误差
    oS.eCache[k] = [1, Ek]  # 更新缓存中第k个样本的预测误差


In [ ]:
def innerLK(i, oS):
    """
    SMO算法的内层循环函数，用于选择第二个变量和执行优化更新

    Args:
    i: int，第一个变量的索引
    oS: optStruct对象，优化结构体

    Returns:
    int，返回1表示成功更新了一对alpha值，返回0表示未能更新
    """
    Ei = calcEk(oS, i)  # 计算第一个变量的预测误差Ei
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        # 检查第一个变量是否违反KKT条件，如果违反则选择第二个变量并执行优化更新
        j, Ej = selectJ(i, oS, Ei)  # 选择第二个变量和计算预测误差Ej
        alphaIold = oS.alphas[i].copy()  # 备份旧的alpha_i
        alphaJold = oS.alphas[j].copy()  # 备份旧的alpha_j
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])  # 计算alpha_j的取值范围下界L
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])  # 计算alpha_j的取值范围上界H
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)  # 计算alpha_j的取值范围下界L
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])  # 计算alpha_j的取值范围上界H
        if L == H:
            print("L==H")
            return 0  # 如果取值范围下界和上界相等，无法进行优化更新，返回0表示未能更新

        eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T  # 计算eta
        if eta >= 0:
            print("eta>=0")
            return 0  # 如果eta大于等于0，无法进行优化更新，返回0表示未能更新

        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta  # 更新alpha_j
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)  # 对alpha_j进行修剪
        updateEk(oS, j)  # 更新第二个变量的预测误差

        if abs(oS.alphas[j] - alphaJold) < 0.00001: 
            print("j not moving enough")
            return 0 # 如果alpha_j的变化量很小，表示未能更新，返回0
            oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])  # 更新alpha_i
        updateEk(oS, i)  # 更新第一个变量的预测误差

        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i, :].T - oS.labelMat[j] * (
                oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T  # 更新b1
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j, :].T - oS.labelMat[j] * (
                oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T  # 更新b2

        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1  # 如果alpha_i在取值范围内，更新b为b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2  # 如果alpha_j在取值范围内，更新b为b2
        else:
            oS.b = (b1 + b2) / 2.0  # 否则更新b为b1和b2的平均值

        return 1  # 成功更新了一对alpha值，返回1
    else:
        return 0  # 未能更新，返回0



In [ ]:
# 定义SMO算法函数，输入参数包括数据矩阵、类别标签、惩罚系数C、容错率tolerance和最大迭代次数maxIter
def smoPK(dataMatIn, classLabels, C, toler, maxIter):
    # 创建optStruct对象oS，该对象保存所有重要的值
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler)
    iter = 0 # 初始化迭代次数iter为0
    entireSet = True # 初始化entireSet变量为True，表示默认遍历整个数据集
    alphaPairsChanged = 0 # 记录alpha是否已经进行了优化
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        # 初始化alphaPairsChanged，在每次循环时设为0，用于记录alpha是否已经进行了优化
        alphaPairsChanged = 0
        # 如果entireSet为True或者alphaPairsChanged不等于0，则遍历整个数据集
        if entireSet:
            for i in range(oS.m): # 遍历所有样本点
                alphaPairsChanged += innerL(i,oS) # 使用innerL选择第二个alpha，并在可能时对其进行优化处理
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1 # 更新迭代次数
        else: # 遍历非边界alpha值，也就是那些不等于0和C的值
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0] # 遍历所有非边界alpha值
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS) # 使用innerL选择第二个alpha，并在可能时对其进行优化处理
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1 # 更新迭代次数
        if entireSet: entireSet = False # 如果本次遍历了整个数据集，则下次将遍历非边界值
        elif (alphaPairsChanged == 0): entireSet = True  # 如果有一次alpha未更新，则遍历整个数据集
        print("iteration number: %d" % iter)
    return oS.b,oS.alphas # 返回训练完毕后的b和alphas
